In [ ]:
!pip install keras

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 데이터 확인
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Dataset 만들기
import keras
from tensorflow.keras.utils import to_categorical

# Detect Face
import cv2
from scipy.ndimage import zoom

# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Importing necessary layers and modules from TensorFlow instead of standalone Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# gpu사용 변수
import tensorflow as tf

device = tf.device('cuda:3')
# device = torch.device('cuda:3') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### ------------------- 훈련 데이터프레임 생성하기 -------------------

In [ ]:
# 전체 이미지에서 얼굴을 찾아내는 함수
def detect_face(frame):

    # cascade pre-trained 모델 불러오기
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # RGB를 gray scale로 바꾸기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # cascade 멀티스케일 분류
    detected_faces = face_cascade.detectMultiScale(gray,
                                                   scaleFactor = 1.1,
                                                   minNeighbors = 6,
                                                   minSize = (shape_x, shape_y),
                                                   flags = cv2.CASCADE_SCALE_IMAGE
                                                  )

    coord = []
    for x, y, w, h in detected_faces:
        if w > 100:
            sub_img = frame[y:y+h, x:x+w]
            coord.append([x, y, w, h])

    return gray, detected_faces, coord

# 전체 이미지에서 찾아낸 얼굴을 추출하는 함수
def extract_face_features(gray, detected_faces, coord, offset_coefficients=(0.075, 0.05)):
    new_face = []
    for det in detected_faces:

        # 얼굴로 감지된 영역
        x, y, w, h = det

        # 이미지 경계값 받기
        horizontal_offset = int(np.floor(offset_coefficients[0] * w))
        vertical_offset = int(np.floor(offset_coefficients[1] * h))

        # gray scacle 에서 해당 위치 가져오기
        extracted_face = gray[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]

        # 얼굴 이미지만 확대
        new_extracted_face = zoom(extracted_face, (shape_x/extracted_face.shape[0], shape_y/extracted_face.shape[1]))
        new_extracted_face = new_extracted_face.astype(np.float32)
        new_extracted_face /= float(new_extracted_face.max()) # sacled
        new_face.append(new_extracted_face)

    return new_face

In [ ]:
origin_data = pd.read_csv('/content/drive/MyDrive/dataset/fer2013.csv')
origin_data.head()

In [ ]:
import pandas as pd
import os
import numpy as np
import cv2  # OpenCV 라이브러리 사용
from sklearn.model_selection import train_test_split

# 각 감정 유형의 디렉토리 이름과 해당하는 레이블
emotions = {
    'angry': 0,
    'disgust': 1,
    'happy': 2,
    'sad': 3,
    'neutral': 4
}

shape_x = 96
shape_y = 96

# 기본 디렉토리 경로
base_dir = "/content/drive/MyDrive/Images/images_Yuri"
base_dir_2 = "/content/drive/MyDrive/Images/images_swan"

# 원본 데이터 로드
origin_data = pd.read_csv('/content/drive/MyDrive/dataset/fer2013.csv')

# emotion 컬럼에서 2와 5를 가진 행 제거
origin_data = origin_data[~origin_data['emotion'].isin([2, 5])]

# emotion 컬럼 값 변경: 3 -> 2, 4 -> 3, 6 -> 4
origin_data['emotion'] = origin_data['emotion'].replace({
    3: 2,
    4: 3,
    6: 4
})

# 이미지를 지정된 크기로 리사이징하는 함수
def resize_image(filepath, new_width=shape_x, new_height=shape_y):
    face = cv2.imread(filepath)
    if face is None:  # 이미지 파일이 없거나 읽을 수 없으면 None 반환
        return None

    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # 흑백으로 변환
    resized_face = cv2.resize(face, (new_width, new_height))  # 크기 조정
    return resized_face.flatten()  # 평탄화하여 반환

# 데이터프레임 생성 함수
def create_emotion_df(base_dir):
    data = {'emotion': [], 'pixels': [], 'Usage': []}

    for emotion, label in emotions.items():
        image_dir = os.path.join(base_dir, emotion)
        image_files = [
            os.path.join(image_dir, file)
            for file in os.listdir(image_dir)
            if file.lower().endswith(('.png', '.jpg', '.jpeg'))
        ]
        for filepath in image_files:
            pixels = resize_image(filepath)  # 이미지 처리
            if pixels is None:  # None 값 건너뛰기
                continue
            data['emotion'].append(label)
            data['pixels'].append(pixels)  # 평탄화된 픽셀 추가
            data['Usage'].append('Training')

    # 데이터프레임 생성
    df = pd.DataFrame(data)

    # Train/Test Split
    train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
    train_df['Usage'] = 'Training'
    test_df['Usage'] = 'PublicTest'

    return pd.concat([train_df, test_df])

# 데이터프레임 생성
final_df = create_emotion_df(base_dir)
final_df_2 = create_emotion_df(base_dir_2)

# pixels 데이터를 공백으로 구분된 문자열로 변환
def flatten_pixels(df):
    df['pixels'] = df['pixels'].apply(lambda x: ' '.join(map(str, x)))
    return df

# pixels 열 변환
final_df = flatten_pixels(final_df)
final_df_2 = flatten_pixels(final_df_2)

# 원본 데이터와 병합
final_df = pd.concat([final_df, final_df_2, origin_data], ignore_index=True)

# None 값 제거
final_df = final_df[final_df['pixels'].apply(lambda x: x is not None)]

# 데이터 확인
print(final_df.head())
print(final_df['pixels'].iloc[0])  # 첫 번째 픽셀 데이터 확인
print(f"Total samples: {len(final_df)}")

In [ ]:
len(final_df['pixels'][0])

In [ ]:
# 각 샘플의 픽셀 데이터 크기 확인
final_df['pixel_length'] = final_df['pixels'].apply(lambda x: len(x.split()) if isinstance(x, str) else len(x))
print(final_df['pixel_length'].value_counts())

In [ ]:
from skimage.transform import resize
import numpy as np

# 픽셀 데이터 크기 변환 함수
def standardize_pixels(pixel_data, current_length, target_size=(96, 96)):
    # 문자열을 numpy 배열로 변환
    if isinstance(pixel_data, str):
        flat_array = np.array(list(map(float, pixel_data.split())))
    else:
        flat_array = np.array(pixel_data)

    # 현재 데이터 크기를 계산
    current_size = int(np.sqrt(current_length))  # ex: 2304 -> 48, 9216 -> 96
    if current_size * current_size != current_length:
        raise ValueError("Invalid pixel data size: not a perfect square")

    # (current_size, current_size)로 리쉐이프
    pixel_array = flat_array.reshape((current_size, current_size))

    # 지정된 크기로 리사이즈
    resized_array = resize(pixel_array, target_size, anti_aliasing=True)
    return resized_array.flatten()  # 다시 1차원으로 변환

# 데이터 변환 함수 적용
def resize_dataset(df):
    df['pixels'] = df.apply(
        lambda row: standardize_pixels(row['pixels'], row['pixel_length']), axis=1
    )
    # (96, 96, 1)로 리쉐이프
    df['pixels'] = df['pixels'].apply(lambda x: np.array(x).reshape((96, 96, 1)))
    return df

# 데이터프레임에 변환 적용
final_df = resize_dataset(final_df)

# 변환된 데이터 검증
print(final_df['pixels'].iloc[0].shape)  # 출력: (96, 96, 1)
print(final_df['pixels'].apply(lambda x: x.size).value_counts())  # 출력: 9216

In [ ]:
# 훈련 데이터와 테스트 데이터 분할
train_df = final_df[final_df['Usage'] == 'Training']
test_df = final_df[final_df['Usage'] == 'PublicTest']

# 데이터 준비
X_train = np.stack(train_df['pixels'].values)
y_train = train_df['emotion'].values
X_test = np.stack(test_df['pixels'].values)
y_test = test_df['emotion'].values

In [ ]:
# 데이터 크기와 샘플 확인
print(f"X_train shape: {X_train.shape}")
print(X_train[0])  # 첫 번째 데이터 확인

In [ ]:
# 4차원 데이터셋 만들기 (데이터개수, x축, y축, rgb)
X_train_ds = np.reshape(X_train, (X_train.shape[0], shape_x, shape_y, 1))
y_train_ds = np.reshape(y_train, (y_train.shape[0], 1))

X_test_ds = np.reshape(X_test, (X_test.shape[0], shape_x, shape_y, 1))
y_test_ds = np.reshape(y_test, (y_test.shape[0], 1))

print(X_train_ds.shape, y_train_ds.shape)
print(X_test_ds.shape, y_test_ds.shape)

In [ ]:
# 데이터타입 float로 변경
train_data = X_train_ds.astype('float32')
test_data = X_test_ds.astype('float32')

# 스케일링
train_data /= 225
test_data /= 225

# y데이터 원핫인코딩
train_labels_onehot = to_categorical(y_train_ds)
test_labels_onehot = to_categorical(y_test_ds)

# input_shape 설정
n_rows, n_cols, n_dims = X_train_ds.shape[1:]
input_shape = (n_rows, n_cols, n_dims)
print(input_shape)

In [ ]:
# 라벨 숫자를 문자로 변경
def get_label(argument):
    labels = {0:'angry', 1:'disgust', 2:'happy', 3:'sad', 4:'neutral'}
    return(labels.get(argument, 'Invalid emotion'))

# 데이터 시각화
plt.figure(figsize=[10,5])

# Train data 중 100번째 이미지
n=150

plt.subplot(121)
plt.imshow(np.squeeze(X_train_ds[n,:,:], axis = 2), cmap='gray')
plt.title("Ground Truth : {}".format(get_label(int(y_train[n]))))

# Test data 중 100번째 이미지
plt.subplot(122)
plt.imshow(np.squeeze(X_test_ds[n,:,:], axis = 2), cmap='gray')
plt.title("Ground Truth : {}".format(get_label(int(y_test[n]))))

In [ ]:
def simple_model():
    model = Sequential()

    # Input layer
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))

    # Add layers
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    # Flatten
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(512, activation='relu'))

    # Output layer : n_classes=5
    model.add(Dense(5, activation='softmax'))

    return model

In [ ]:
model = simple_model()
model.summary()

In [ ]:
# 학습
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 데이터 증강
datagen = ImageDataGenerator(zoom_range=0.2,          # 랜덤하게 이미지 줌 하는 비율
                             rotation_range=10,       # 램덤하게 이미지 회전하는 비율 (0도~180도)
                             width_shift_range=0.1,   # 랜덤하게 이미지 가로로 이동하는 비율
                             height_shift_range=0.1,  # 랜덤하게 이미지 세로로 이동하는 비율
                             horizontal_flip=True,    # 랜덤하게 이미지 수평 뒤집기
                             vertical_flip=False)     # 랜덤하게 이미지 수직 뒤집기

# 모델 학습을 위한 파라미터 설정
batch_size = 32
n_epochs = 10
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# history = model.fit_generator(datagen.flow(train_data, train_labels_onehot, batch_size=batch_size),
#                               steps_per_epoch=int(np.ceil(train_data.shape[0]/float(batch_size))),
#                               epochs=n_epochs,
#                               validation_data=(test_data, test_labels_onehot)
#                              )

history = model.fit(
    datagen.flow(train_data, train_labels_onehot, batch_size=batch_size),
    steps_per_epoch=int(np.ceil(train_data.shape[0] / float(batch_size))),
    epochs=n_epochs,
    validation_data=(test_data, test_labels_onehot)  # Ensure this is included
)

In [ ]:
# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=2.0)
plt.plot(history.history['val_loss'],'b',linewidth=2.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=2.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
model_path = '/content/drive/MyDrive/Model/sad.keras'  # Update the path as needed
model.save(model_path)

In [ ]:
# 원본이미지 확인
face = cv2.imread('/content/drive/MyDrive/dataset/disgust_6.jpg')

# 얼굴 추출
gray, detected_faces, coord = detect_face(face)
face_zoom = extract_face_features(gray, detected_faces, coord)

# 모델 추론
input_data = np.reshape(face_zoom[0].flatten(), (1, 96, 96, 1))
output_data = model.predict(input_data)
print(len(output_data[0]))

result = np.argmax(output_data)

# 결과 문자로 변환
if result == 0:
    emotion = 'angry'
elif result == 1:
    emotion = 'disgust'
elif result == 2:
    emotion = 'happy'
elif result == 3:
    emotion = 'sad'
elif result == 4:
    emotion = 'neutral'

# 시각화
plt.subplot(121)
plt.title("Original Face")
plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title(f"Extracted Face : {emotion}")
plt.imshow(face_zoom[0])

In [ ]:
# 원본이미지 확인
face = cv2.imread('/content/drive/MyDrive/Images/images_Yuri/disgust/IMG_2584.jpg')

# 얼굴 추출
gray, detected_faces, coord = detect_face(face)
face_zoom = extract_face_features(gray, detected_faces, coord)

# 모델 추론
input_data = np.reshape(face_zoom[0].flatten(), (1, 96, 96, 1))
output_data = model.predict(input_data)
print("angry :", output_data[0][0], "disgust :", output_data[0][1], "happy :", output_data[0][2], "sad :", output_data[0][3], "neutral :", output_data[0][4])


result = np.argmax(output_data)

# 결과 문자로 변환
if result == 0:
    emotion = 'angry'
elif result == 1:
    emotion = 'disgust'
elif result == 2:
    emotion = 'happy'
elif result == 3:
    emotion = 'sad'
elif result == 4:
    emotion = 'neutral'

# 시각화
plt.subplot(121)
plt.title("Original Face")
plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title(f"Extracted Face : {emotion}")
plt.imshow(face_zoom[0])

In [ ]:
# 원본이미지 확인
face = cv2.imread('/content/drive/MyDrive/Images/images_tmddus/happy/image_412.jpg')

# 얼굴 추출
gray, detected_faces, coord = detect_face(face)
face_zoom = extract_face_features(gray, detected_faces, coord)

# 모델 추론
input_data = np.reshape(face_zoom[0].flatten(), (1, 96, 96, 1))
output_data = model.predict(input_data)
print("angry :", output_data[0][0], "disgust :", output_data[0][1], "happy :", output_data[0][2], "sad :", output_data[0][3], "neutral :", output_data[0][4])

result = np.argmax(output_data)

# 결과 문자로 변환
if result == 0:
    emotion = 'angry'
elif result == 1:
    emotion = 'disgust'
elif result == 2:
    emotion = 'happy'
elif result == 3:
    emotion = 'sad'
elif result == 4:
    emotion = 'neutral'

# 시각화
plt.subplot(121)
plt.title("Original Face")
plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title(f"Extracted Face : {emotion}")
plt.imshow(face_zoom[0])

In [ ]:
# 원본이미지 확인
face = cv2.imread('/content/drive/MyDrive/Images/images_tmddus/happy/image_501.jpg')

# 얼굴 추출
gray, detected_faces, coord = detect_face(face)
face_zoom = extract_face_features(gray, detected_faces, coord)

# 모델 추론
input_data = np.reshape(face_zoom[0].flatten(), (1, 96, 96, 1))
output_data = model.predict(input_data)
print("angry :", output_data[0][0], "disgust :", output_data[0][1], "happy :", output_data[0][2], "sad :", output_data[0][3], "neutral :", output_data[0][4])

result = np.argmax(output_data)

# 결과 문자로 변환
if result == 0:
    emotion = 'angry'
elif result == 1:
    emotion = 'disgust'
elif result == 2:
    emotion = 'happy'
elif result == 3:
    emotion = 'sad'
elif result == 4:
    emotion = 'neutral'

# 시각화
plt.subplot(121)
plt.title("Original Face")
plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title(f"Extracted Face : {emotion}")
plt.imshow(face_zoom[0])

In [ ]:
# 원본이미지 확인
face = cv2.imread('/content/drive/MyDrive/Images/images_soo/happy/image_110.jpg')

# 얼굴 추출
gray, detected_faces, coord = detect_face(face)
face_zoom = extract_face_features(gray, detected_faces, coord)

# 모델 추론
input_data = np.reshape(face_zoom[0].flatten(), (1, 96, 96, 1))
output_data = model.predict(input_data)
print("angry :", output_data[0][0], "disgust :", output_data[0][1], "happy :", output_data[0][2], "sad :", output_data[0][3], "neutral :", output_data[0][4])

result = np.argmax(output_data)

# 결과 문자로 변환
if result == 0:
    emotion = 'angry'
elif result == 1:
    emotion = 'disgust'
elif result == 2:
    emotion = 'happy'
elif result == 3:
    emotion = 'sad'
elif result == 4:
    emotion = 'neutral'

# 시각화
plt.subplot(121)
plt.title("Original Face")
plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title(f"Extracted Face : {emotion}")
plt.imshow(face_zoom[0])

In [ ]:
# 원본이미지 확인
face = cv2.imread('/content/drive/MyDrive/Images/images_swan/happy/image_300.jpg')

# 얼굴 추출
gray, detected_faces, coord = detect_face(face)
face_zoom = extract_face_features(gray, detected_faces, coord)

# 모델 추론
input_data = np.reshape(face_zoom[0].flatten(), (1, 96, 96, 1))
output_data = model.predict(input_data)
print("angry :", output_data[0][0], "disgust :", output_data[0][1], "happy :", output_data[0][2], "sad :", output_data[0][3], "neutral :", output_data[0][4])

result = np.argmax(output_data)

# 결과 문자로 변환
if result == 0:
    emotion = 'angry'
elif result == 1:
    emotion = 'disgust'
elif result == 2:
    emotion = 'happy'
elif result == 3:
    emotion = 'sad'
elif result == 4:
    emotion = 'neutral'

# 시각화
plt.subplot(121)
plt.title("Original Face")
plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title(f"Extracted Face : {emotion}")
plt.imshow(face_zoom[0])

In [ ]:
# 원본이미지 확인
face = cv2.imread('/content/drive/MyDrive/Images/images_Yuri/happy/angry_651.jpg')

# 얼굴 추출
gray, detected_faces, coord = detect_face(face)
face_zoom = extract_face_features(gray, detected_faces, coord)

# 모델 추론
input_data = np.reshape(face_zoom[0].flatten(), (1, 96, 96, 1))
output_data = model.predict(input_data)
print("angry :", output_data[0][0], "disgust :", output_data[0][1], "happy :", output_data[0][2], "sad :", output_data[0][3], "neutral :", output_data[0][4])

result = np.argmax(output_data)

# 결과 문자로 변환
if result == 0:
    emotion = 'angry'
elif result == 1:
    emotion = 'disgust'
elif result == 2:
    emotion = 'happy'
elif result == 3:
    emotion = 'sad'
elif result == 4:
    emotion = 'neutral'

# 시각화
plt.subplot(121)
plt.title("Original Face")
plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

plt.subplot(122)
plt.title(f"Extracted Face : {emotion}")
plt.imshow(face_zoom[0])

In [ ]:
# real-time video demo

import cv2
path = '/Users/daisy/Desktop/2022-13750/3학년 2학기/감정트레이닝/haarcascade_frontalface_default.xml'
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

rectangle_bgr = (255, 255, 255)
img = np.array((500, 500))

text = 'Some text in the box'
(text_width, text_height) = cv2.getTextSize(text, font, fontScale = font_scale, thickness = 1)[0]
text_offset_x = 10
text_offset_y = img.shape[0] - 25

box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height - 2))
cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale=font_scale, color = (0, 0, 0), thickness=1)

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError('Cannot open webcam')

while True:
    ret, frame = cap.read()
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + path)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    for x, y, w, h in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        facess = faceCascade.detectMultiScale(roi_gray)
        if len(facess) == 0:
            print('Face not detected')
        else:
            for (ex, ey, ew, eh) in facess:
                face_roi = roi_color[ey:ey+eh, ex:ex+ew]

    final_image = cv2.resize(face_roi, (224, 224))
    final_image = np.expand_dims(final_image, axis = 0)
    final_image = final_image / 255.0

    font = cv2.FONT_HERSHEY_SIMPLEX

    Predictions = new_model.predict(final_image)

    font_scale = 1.5
    font = cv2.FONT_HERSHEY_PLAIN

    # 여기서부터 다양한 감정으로 조정하면 될 것 같음
    if (np.argmax(Predictions) == 0):
        status = 'Angry'

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1+int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

    elif (np.argmax(Predictions)==1):
        status = 'Disgust'

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1+int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

    elif (np.argmax(Predictions)==2):
        status = 'Fear'

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1+int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

    elif (np.argmax(Predictions)==3):
        status = 'Happy'

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1+int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

    elif (np.argmax(Predictions)==4):
        status = 'Sad'

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1+int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))


    elif (np.argmax(Predictions)==5):
        status = 'Surprise'

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1+int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

    else:
        status = 'Neutral'

        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, x1), (x1+w1, y1+h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1+int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

    cv2.imshow('Face Emotion Recognition', frame)
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
pip install opencv-python
pip install tensorflow  # 또는 keras, 모델에 따라 다름

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# 모델 로드
model_path = '/content/drive/MyDrive/Model/tmddus.keras'
model = load_model(model_path)

# 얼굴 검출을 위한 OpenCV 전처리
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# 감정 레이블
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def get_face(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    if len(faces) == 0:
        return None, None
    (x, y, w, h) = faces[0]
    return gray[y:y+h, x:x+w], faces[0]

def analyze_emotion(face_image):
    face_image = cv2.resize(face_image, (48, 48))
    face_image = face_image.reshape((1, 48, 48, 1))
    face_image = face_image.astype('float32') / 255
    return model.predict(face_image)

# 카메라 캡처 시작
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    face, bbox = get_face(frame)
    if face is not None:
        output_data = analyze_emotion(face)
        emotion = emotion_labels[np.argmax(output_data)]

        # 결과 표시
        x, y, w, h = bbox
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imshow('Video Feed', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# ---------------------- test -----------------------

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 데이터 확인
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Dataset 만들기
import keras
from tensorflow.keras.utils import to_categorical

# Detect Face
import cv2
from scipy.ndimage import zoom

# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Importing necessary layers and modules from TensorFlow instead of standalone Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 전체 이미지에서 얼굴을 찾아내는 함수
def detect_face(frame):

    # cascade pre-trained 모델 불러오기
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # RGB를 gray scale로 바꾸기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # cascade 멀티스케일 분류
    detected_faces = face_cascade.detectMultiScale(gray,
                                                   scaleFactor = 1.1,
                                                   minNeighbors = 6,
                                                   minSize = (shape_x, shape_y),
                                                   flags = cv2.CASCADE_SCALE_IMAGE
                                                  )

    coord = []
    for x, y, w, h in detected_faces:
        if w > 100:
            sub_img = frame[y:y+h, x:x+w]
            coord.append([x, y, w, h])

    return gray, detected_faces, coord

# 전체 이미지에서 찾아낸 얼굴을 추출하는 함수
def extract_face_features(gray, detected_faces, coord, offset_coefficients=(0.075, 0.05)):
    new_face = []
    for det in detected_faces:

        # 얼굴로 감지된 영역
        x, y, w, h = det

        # 이미지 경계값 받기
        horizontal_offset = int(np.floor(offset_coefficients[0] * w))
        vertical_offset = int(np.floor(offset_coefficients[1] * h))

        # gray scacle 에서 해당 위치 가져오기
        extracted_face = gray[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]

        # 얼굴 이미지만 확대
        new_extracted_face = zoom(extracted_face, (shape_x/extracted_face.shape[0], shape_y/extracted_face.shape[1]))
        new_extracted_face = new_extracted_face.astype(np.float32)
        new_extracted_face /= float(new_extracted_face.max()) # sacled
        new_face.append(new_extracted_face)

    return new_face

In [ ]:
shape_x = 96
shape_y = 96

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import io

# 모델 로드
model_path = '/content/drive/MyDrive/Model/sad.keras'  # Update the path as needed
model = load_model(model_path)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

filename = take_photo()
print("Image saved to:", filename)

In [ ]:
emotion_labels = ['angry', 'disgust', 'happy', 'sad', 'neutral']  # Ensure this aligns with the model's output

import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

model_path = '/content/drive/MyDrive/Model/sad.keras'  # Update the path as needed
model = load_model(model_path)

def analyze_emotion(image_path, model):
    # Load the original image
    face = cv2.imread(image_path)

    # Extract face
    gray, detected_faces, coord = detect_face(face)
    if len(detected_faces) == 0:
        print("No face detected in the image.")
        return
    face_zoom = extract_face_features(gray, detected_faces, coord)

    # Convert the extracted face into the format required by the model
    img_array = np.reshape(face_zoom[0], (1, 96, 96, 1))

    # Model prediction
    prediction = model.predict(img_array)[0]  # Get the probabilities for each emotion

    print(prediction)

    # Convert prediction to percentages
    prediction_percentages = {emotion_labels[i]: round(prob * 100, 10) for i, prob in enumerate(prediction)}

    # Print the probabilities for each emotion
    print("\nPrediction Percentages:")
    for emotion, percentage in prediction_percentages.items():
        print(f"{emotion}: {percentage}%")

    # Display the original and extracted face
    plt.subplot(121)
    plt.title("Original Face")
    plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

    plt.subplot(122)
    plt.title("Extracted Face")
    plt.imshow(face_zoom[0], cmap='gray')
    plt.axis('off')
    plt.show()

# Example usage
analyze_emotion(filename, model)

In [ ]:
print(f"Min pixel value: {face_zoom[0].min()}, Max pixel value: {face_zoom[0].max()}")

In [ ]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model

# Emotion labels
emotion_labels = ['angry', 'disgust', 'happy', 'sad', 'neutral']

# Load the pre-trained model
model_path = '/content/drive/MyDrive/Model/sad.keras'  # 모델 경로
model = load_model(model_path)

# Define the base directory for the folders
base_dir = "/content/drive/MyDrive/Images/images_sohn"  # 폴더 경로

# Function to process a single image and get predictions
def get_emotion_prediction(image_path, model):
    # Load the image
    face = cv2.imread(image_path)
    if face is None:
        print(f"Error loading image: {image_path}")
        return None

    # Extract the face
    gray, detected_faces, coord = detect_face(face)
    if len(detected_faces) == 0:
        print(f"No face detected in: {image_path}")
        return None
    face_zoom = extract_face_features(gray, detected_faces, coord)
    if face_zoom[0].shape != (96, 96):
        face_zoom[0] = cv2.resize(face_zoom[0], (96, 96))  # Ensure size is correct

    # Prepare the image for the model
    img_array = np.reshape(face_zoom[0], (1, 96, 96, 1)).astype('float32') / 255.0

    # Predict the emotion probabilities
    prediction = model.predict(img_array)[0]  # Shape: (5,)
    return prediction

# Dictionary to store predictions for each folder
emotion_predictions = {label: [] for label in emotion_labels}

# Loop through each "extra_" folder
for folder_name in os.listdir(base_dir):
    if not folder_name.startswith("241120_"):  # Only process folders starting with "extra_"
        continue

    folder_path = os.path.join(base_dir, folder_name)
    if not os.path.isdir(folder_path):
        continue

    # Process each image in the folder
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        prediction = get_emotion_prediction(image_path, model)
        if prediction is not None:
            emotion_predictions[folder_name.split('_')[-1]].append(prediction)  # Save prediction

# Calculate the variance sum for each emotion
variance_sum = {}
for emotion, predictions in emotion_predictions.items():
    if len(predictions) > 0:
        predictions = np.array(predictions)  # Convert to NumPy array
        variance_sum[emotion] = np.sum(np.var(predictions, axis=0))  # Variance sum across predictions

# Print the variance sum for each emotion
print("\nVariance Sum for Each Emotion:")
for emotion, var_sum in variance_sum.items():
    print(f"{emotion}: {var_sum}")

In [ ]:
total_variance_sum = sum(variance_sum.values())
print('Total variance:', total_variance_sum)

# 완성

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 데이터 확인
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Dataset 만들기
import keras
from tensorflow.keras.utils import to_categorical

# Detect Face
import cv2
from scipy.ndimage import zoom

# Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Importing necessary layers and modules from TensorFlow instead of standalone Keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 전체 이미지에서 얼굴을 찾아내는 함수
def detect_face(frame):

    # cascade pre-trained 모델 불러오기
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # RGB를 gray scale로 바꾸기
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # cascade 멀티스케일 분류
    detected_faces = face_cascade.detectMultiScale(gray,
                                                   scaleFactor = 1.1,
                                                   minNeighbors = 6,
                                                   minSize = (shape_x, shape_y),
                                                   flags = cv2.CASCADE_SCALE_IMAGE
                                                  )

    coord = []
    for x, y, w, h in detected_faces:
        if w > 100:
            sub_img = frame[y:y+h, x:x+w]
            coord.append([x, y, w, h])

    return gray, detected_faces, coord

# 전체 이미지에서 찾아낸 얼굴을 추출하는 함수
def extract_face_features(gray, detected_faces, coord, offset_coefficients=(0.075, 0.05)):
    new_face = []
    for det in detected_faces:

        # 얼굴로 감지된 영역
        x, y, w, h = det

        # 이미지 경계값 받기
        horizontal_offset = int(np.floor(offset_coefficients[0] * w))
        vertical_offset = int(np.floor(offset_coefficients[1] * h))

        # gray scacle 에서 해당 위치 가져오기
        extracted_face = gray[y+vertical_offset:y+h, x+horizontal_offset:x-horizontal_offset+w]

        # 얼굴 이미지만 확대
        new_extracted_face = zoom(extracted_face, (shape_x/extracted_face.shape[0], shape_y/extracted_face.shape[1]))
        new_extracted_face = new_extracted_face.astype(np.float32)
        new_extracted_face /= float(new_extracted_face.max()) # sacled
        new_face.append(new_extracted_face)

    return new_face

shape_x = 96
shape_y = 96

from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from PIL import Image
import io

# 모델 로드
model_path = '/content/drive/MyDrive/Model/sad.keras'  # Update the path as needed
model = load_model(model_path)
emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = 'Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            // Wait for Capture to be clicked.
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

emotion_labels = ['angry', 'disgust', 'happy', 'sad', 'neutral']  # Ensure this aligns with the model's output

import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

model = load_model(model_path)

def analyze_emotion(image_path, model):
    # Load the original image
    face = cv2.imread(image_path)

    # Extract face
    gray, detected_faces, coord = detect_face(face)
    if len(detected_faces) == 0:
        print("No face detected in the image.")
        return
    face_zoom = extract_face_features(gray, detected_faces, coord)

    # Convert the extracted face into the format required by the model
    img_array = np.reshape(face_zoom[0], (1, 96, 96, 1))

    # Model prediction
    prediction = model.predict(img_array)[0]  # Get the probabilities for each emotion

    # Convert prediction to percentages
    prediction_percentages = {emotion_labels[i]: round(prob * 100, 10) for i, prob in enumerate(prediction)}

    # Display the original and extracted face
    plt.subplot(121)
    plt.title("Original Face")
    plt.imshow(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))

    plt.subplot(122)
    plt.title("Extracted Face")
    plt.imshow(face_zoom[0], cmap='gray')
    plt.axis('off')
    plt.show()

    return prediction

In [ ]:
while True:
        try:
            print("Show your happy face! Do not exaggerate and just show your natural happy face.")
            happy = take_photo()
            print("Show your happy face! Do not exaggerate and just show your natural happy face.")
            filename = take_photo()
            print("Photo taken! Analyzing your neutral face type...")

            prediction = analyze_emotion(filename, model)

            if prediction[3] >= 0.25:
                if prediction[0] < 0.15:
                    neutral = 'SAD'
                    model = 1
                else:
                    # SAD, angry 동시 충족할 시 기준점에서 떨어진 정도로 결정
                    if prediction[0] - 0.15 > prediction[3] - 0.25:
                        neutral = 'ANGRY'
                        model = 2
                    else:
                        neutral = 'SAD'
                        model = 1
            elif prediction[0] >= 0.15:
                neutral = 'ANGRY'
                model = 2
            else:
                neutral = 'HAPPY'
                model = 3

            print("Finished analyzing! Now move to the next room and go to the model number", model)
            break  # 성공적으로 실행되었으므로 루프 종료

        except Exception as e:
            print("An error occurred:", str(e))
            print("Retrying...")